In [4]:
import re 
import urllib.request as urllib_request
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from utils.functions import *

In [5]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

## Rotina de scraping
### Declara-se a variável cards
cards = []

###
url = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0'}
headers = {'User-Agent': 'xxxx'}
req = Request(url, headers = headers)
#req = Request(url)
response = urlopen(req)
html = response.read()
html = html.decode('utf-8')
html = trata_html(html)


In [6]:
## Criação de um objeto Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())
tickets = soup.findAll('span', {'class': 'ticker'})
codigos = []
for ticket in tickets:
    codigos.append(ticket.get_text().lower())

In [7]:
codigo = codigos[0]
url = 'https://fiis.com.br/' + codigo
headers = {'User-Agent': 'xxxx'} # olhar no seu PC
req = Request(url, headers = headers)
response = urlopen(req)
html = response.read()
html = html.decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')
items = soup.findAll('span', {'class': 'value'})
items = soup.find_all(class_ = 'value')
infos = []
card = {}
for item in items:
    infos.append(item.get_text())

In [8]:
for codigo in codigos[0:10]:
    url = 'https://fiis.com.br/' + codigo
    headers = {'User-Agent': 'xxxx'} # olhar no seu PC
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()
    html = html.decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all(class_ = 'value')
    infos = []
    card = {}
    for item in items:
        infos.append(item.get_text())
    card['dividend_yield'] = infos[0]
    card['ultimo_rendimento'] = infos[1]
    card['patrimonio_liquido'] = infos[2]
    card['valor_patrimonial_por_cota'] = infos[3]
    card['telefone'] = infos[4]
    card['email'] = infos[5]
    card['site'] = infos[6]
    card['name'] = infos[7]
    card['tipo_FII'] = infos[8]
    card['tipo_ANBIMA'] = infos[9]
    card['registro_CVM'] = infos[10]
    card['numero_cotas'] = infos[11]
    card['numero_cotistas'] = infos[12]
    card['CNPJ'] = infos[13]
    card['cotacao'] = infos[14]
    card['min_52_weeks'] = infos[15]
    card['max_52_weeks'] = infos[16]
    card['return_12_months'] = infos[17]
    card['ticker'] = codigo.upper()
    
    admin = soup.findAll("div", {"class": "text-wrapper"})
    card['Administrador'] = re.sub("ADMINISTRADOR|[0-9].*[0-9]", "", admin[0].get_text())
    
    # cotacao = soup.findAll("div", {"class": "item quotation"})
    # cotacao = cotacao[0].find("span", {"class": "value"}).get_text()
    
    # min52 = soup.findAll("div", {"class": "item min52"})
    # min52 = min52[0].find("div", {"class": "value"}).get_text()

    # max52 = soup.findAll("div", {"class": "item max52"})
    # max52 = max52[0].find("div", {"class": "value"}).get_text()

    # valorizacao12 = soup.findAll("div", {"class": "item val12"})
    # valorizacao12 = valorizacao12[0].find("div", {"class": "value"}).get_text()
    
    # card["Cotacao Atual"] = cotacao
    # card["Min. 52 semanas"] = min52
    # card["Max. 52 semanas"] = max52
    # card["Valorizacao"] = valorizacao12
    
    cards.append(card)

In [13]:
## Cria-se um dataframe com os resultados
dataset = pd.DataFrame(cards)

In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   dividend_yield              10 non-null     object
 1   ultimo_rendimento           10 non-null     object
 2   patrimonio_liquido          10 non-null     object
 3   valor_patrimonial_por_cota  10 non-null     object
 4   telefone                    10 non-null     object
 5   email                       10 non-null     object
 6   site                        10 non-null     object
 7   name                        10 non-null     object
 8   tipo_FII                    10 non-null     object
 9   tipo_ANBIMA                 10 non-null     object
 10  registro_CVM                10 non-null     object
 11  numero_cotas                10 non-null     object
 12  numero_cotistas             10 non-null     object
 13  CNPJ                        10 non-null     object
 1

In [15]:
dataset

,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_FII,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,ticker,Administrador
0,"0,89%","R$0,65","R$1,12 B","R$91,83",(11) 3509-6600,[email protected],www.riobravo.com.br,FII ABC IMOB,Tijolo: Shoppings,Renda Gestão Passiva,15/05/1996,12.203.832,19.040,01.201.140/0001-90,"72,80","R$60,01","R$78,27","5,02%",ABCP11,Rio Bravo Investimentos
1,"1,37%","R$1,35","R$172,46 M","R$96,47",21 3262-9600,[email protected],www.btgpactual.com,FII AFHI CRI,Papel: CRIs,Títulos e Valores Mobiliários Gestão Ativa,20/10/2020,1.787.671,7.700,36.642.293/0001-58,"98,25","R$88,00","R$103,45","6,79%",AFHI11,BTG Pactual
2,"1,28%","R$1,14","R$70,55 M","R$102,07",(11)31330350,[email protected],www.brltrust.com.br,FII ALIANZFF,Papel: Fundo de Fundos,Títulos e Valores Mobiliários Gestão Ativa,22/01/2020,691.192,2.584,34.847.063/0001-08,"88,80","R$75,05","R$300,00","60,36%",AFOF11,BRL Trust DTVM
3,"0,91%","R$0,72","R$491,83 M","R$101,93",3223-7700,[email protected],www.modal.com.br,FII AUTONOMY,Tijolo: Escritórios,Renda Gestão Ativa,24/08/2020,4.824.987,6.762,35.765.826/0001-26,"78,99","R$67,50","R$88,00","10,23%",AIEC11,MAF DTVM
4,"0,04%","R$0,37","R$250,49 M","R$2.253,05",21 3262-9600,[email protected],www.btgpactual.com,FII TORRE AL,Tijolo: Escritórios,Renda Gestão Passiva,03/01/2006,111.177,3.114,07.122.725/0001-00,"932,03","R$830,00","R$1.300,00","18,23%",ALMI11,BTG Pactual
5,"0,67%","R$0,78","R$642,32 M","R$108,46",21 3262-9600,[email protected],www.btgpactual.com,FII ALIANZA,Tijolo: Híbrido,Renda Gestão Ativa,20/10/2017,5.922.011,98.531,28.737.771/0001-85,"115,43","R$101,51","R$129,88","7,66%",ALZR11,BTG Pactual
6,"1,15%","R$40,15","R$1,54 B","R$3.403,44",(11) 35248888,[email protected],https://www.genialinvestimentos.com.br/,FII ANCAR IC,Tijolo: Shoppings,Desenvolvimento para Renda Gestão Ativa,11/04/2011,451.113,57,07.789.135/0001-27,"3.500,00","R$3.366,00","R$3.500,00","3,98%",ANCR11B,Genial Investimentos Corretora de Valores
7,"1,20%","R$0,12","R$39,01 M","R$9,72",(11)31330350,[email protected],www.brltrust.com.br,FII NAVI RSD,Híbrido (Tijolo/Papel),Híbrido Gestão Ativa,10/08/2021,4.012.857,837,42.432.327/0001-82,"10,02","R$9,00","R$11,53","5,11%",APTO11,BRL Trust DTVM
8,"1,93%","R$2,02","R$288,47 M","R$100,21",21 3262-9600,[email protected],www.btgpactual.com,FII ARCTIUM,Tijolo: Galpões,Híbrido Gestão Ativa,31/07/2017,2.878.568,11.482,28.267.696/0001-36,"104,42","R$95,37","R$112,00","5,07%",ARCT11,BTG Pactual
9,"1,36%","R$1,28","R$96,61 M","R$92,16",(21) 3514-0000,[email protected],www.oliveiratrust.com.br,FII ATRIO,Papel: CRIs,Títulos e Valores Mobiliários Gestão Ativa,01/07/2019,1.048.320,6.878,32.006.821/0001-21,"94,50","R$89,36","R$114,00","6,62%",ARRI11,Oliveira Trust


In [16]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
#dataset = dataset.apply(lambda x: x.str.replace("R\\$|\\.","", regex = True).str.replace(",", ".").astype(float) if x.name in COLUMNS_LIST else x)
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset["p_vp"] = dataset['cotacao'] / dataset['valor_patrimonial_por_cota']
#dataset.to_csv('C:/Users/marco/Desktop/Estatistica/fiis_dataset.csv', sep = ';', index = False)


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   dividend_yield              10 non-null     object 
 1   ultimo_rendimento           10 non-null     float64
 2   patrimonio_liquido          10 non-null     object 
 3   valor_patrimonial_por_cota  10 non-null     float64
 4   telefone                    10 non-null     object 
 5   email                       10 non-null     object 
 6   site                        10 non-null     object 
 7   name                        10 non-null     object 
 8   tipo_FII                    10 non-null     object 
 9   tipo_ANBIMA                 10 non-null     object 
 10  registro_CVM                10 non-null     object 
 11  numero_cotas                10 non-null     float64
 12  numero_cotistas             10 non-null     float64
 13  CNPJ                        10 non-nul